In [1]:
import os
import glob
import pandas as pd
import numpy as np
import warnings
import xlsxwriter
warnings.filterwarnings('ignore')

In [2]:
path = r'raw_data'

sub = 0

csv_files = glob.glob(os.path.join(path, "*.csv"))

exclude_ID = '0000000000'

all_simi = pd.DataFrame()
all_classi = pd.DataFrame()
all_mani = pd.DataFrame()

for file in csv_files:
    each_df = pd.read_csv(file)
    
    excluded =  each_df[each_df['PROLIFIC_ID*'].str.contains(exclude_ID)]
    
    if len(excluded) > 0:
        continue

    #drop columns
    cols = [3,4,5,6,7,8,10,11,12,13,14,15,18,19,20,21,22,29,30,31,32,33,35,36]
    each_df.drop(each_df.columns[[cols]],axis=1,inplace=True)
    
    #copy glasses name
    items = each_df.loc[0,'glasses1'].split('_')
    each_df.insert(0,'pair_assigned', items[0] + '_' + items[1])
    each_df['condition'] = each_df.loc[0,'condition']
    each_df['glasses1'] = each_df.loc[0,'glasses1'].split('_')[0]
    each_df['glasses2'] = each_df.loc[0,'glasses2'].split('_')[1]
    each_df.rename(columns={'PROLIFIC_ID*': 'ID'}, inplace=True)
    
    simi_df = each_df.iloc[:,0:10].dropna(subset=['similarity_resp.keys'])
    simi_df = simi_df.astype({'glasses_left_num': int, 'glasses_right_num': int})
    classi_df = pd.concat([each_df.iloc[:,0:5], each_df.iloc[:,10:13]], axis=1).dropna(subset=['class_resp.keys'])
    mani_df = each_df[['ID','pair_assigned','condition','manipulation_check']].iloc[-1]
    
    
    
    simi_df['glasses_choose'] = ' '
    simi_df['correct'] = 0
    simi_df['pair'] = ' '
    
    """
    for i in simi_df.index:
        if simi_df['ask_glasses'][i] == 'glasses1':
            simi_df['ask_glasses'][i] = simi_df['glasses1'][i]
        elif simi_df['ask_glasses'][i] == 'glasses2':
            simi_df['ask_glasses'][i] = simi_df['glasses2'][i]
    """
    
    for i in simi_df.index:
        if simi_df['similarity_resp.keys'][i] == 'x':
            simi_df['glasses_choose'][i] = simi_df['glasses_left_num'][i]
        elif simi_df['similarity_resp.keys'][i] == 'period':
            simi_df['glasses_choose'][i] = simi_df['glasses_right_num'][i]
            
    for i in simi_df.index:
        if simi_df['ask_glasses'][i] == 'glasses1':
            if simi_df['similarity_resp.keys'][i] == 'x':
                if simi_df['glasses_left_num'][i] < simi_df['glasses_right_num'][i]:
                    simi_df['correct'][i] = 1
            elif simi_df['similarity_resp.keys'][i] == 'period':
                if simi_df['glasses_right_num'][i] < simi_df['glasses_left_num'][i]:
                    simi_df['correct'][i] = 1
        elif simi_df['ask_glasses'][i] == 'glasses2':
            if simi_df['similarity_resp.keys'][i] == 'x':
                if simi_df['glasses_left_num'][i] > simi_df['glasses_right_num'][i]:
                    simi_df['correct'][i] = 1
            elif simi_df['similarity_resp.keys'][i] == 'period':
                if simi_df['glasses_right_num'][i] > simi_df['glasses_left_num'][i]:
                    simi_df['correct'][i] = 1
                    
    for i in simi_df.index:
        if simi_df['glasses_left_num'][i] == 0 and simi_df['glasses_right_num'][i] == 2:
            simi_df['pair'][i] = 'pair1'
        elif simi_df['glasses_left_num'][i] == 1 and simi_df['glasses_right_num'][i] == 3:
            simi_df['pair'][i] = 'pair2'
        elif simi_df['glasses_left_num'][i] == 2 and simi_df['glasses_right_num'][i] == 4:
            simi_df['pair'][i] = 'pair3'
        elif simi_df['glasses_left_num'][i] == 3 and simi_df['glasses_right_num'][i] == 5:
            simi_df['pair'][i] = 'pair4'
        elif simi_df['glasses_left_num'][i] == 4 and simi_df['glasses_right_num'][i] == 6:
            simi_df['pair'][i] = 'pair5'
        elif simi_df['glasses_left_num'][i] == 5 and simi_df['glasses_right_num'][i] == 7:
            simi_df['pair'][i] = 'pair6'
        elif simi_df['glasses_left_num'][i] == 6 and simi_df['glasses_right_num'][i] == 8:
            simi_df['pair'][i] = 'pair7'
        elif simi_df['glasses_left_num'][i] == 7 and simi_df['glasses_right_num'][i] == 9:
            simi_df['pair'][i] = 'pair8'
        elif simi_df['glasses_left_num'][i] == 8 and simi_df['glasses_right_num'][i] == 10:
            simi_df['pair'][i] = 'pair9'
            
        elif simi_df['glasses_right_num'][i] == 0 and simi_df['glasses_left_num'][i] == 2:
            simi_df['pair'][i] = 'pair1'
        elif simi_df['glasses_right_num'][i] == 1 and simi_df['glasses_left_num'][i] == 3:
            simi_df['pair'][i] = 'pair2'
        elif simi_df['glasses_right_num'][i] == 2 and simi_df['glasses_left_num'][i] == 4:
            simi_df['pair'][i] = 'pair3'
        elif simi_df['glasses_right_num'][i] == 3 and simi_df['glasses_left_num'][i] == 5:
            simi_df['pair'][i] = 'pair4'
        elif simi_df['glasses_right_num'][i] == 4 and simi_df['glasses_left_num'][i] == 6:
            simi_df['pair'][i] = 'pair5'
        elif simi_df['glasses_right_num'][i] == 5 and simi_df['glasses_left_num'][i] == 7:
            simi_df['pair'][i] = 'pair6'
        elif simi_df['glasses_right_num'][i] == 6 and simi_df['glasses_left_num'][i] == 8:
            simi_df['pair'][i] = 'pair7'
        elif simi_df['glasses_right_num'][i] == 7 and simi_df['glasses_left_num'][i] == 9:
            simi_df['pair'][i] = 'pair8'
        elif simi_df['glasses_right_num'][i] == 8 and simi_df['glasses_left_num'][i] == 10:
            simi_df['pair'][i] = 'pair9'
            
    
    classi_df['class_glasses'] = 15
    classi_df['glasses_choose'] = 2
    
    for i in classi_df.index:
        if classi_df['class_resp.keys'][i] == 'x':
            classi_df['glasses_choose'][i] = 0
        elif classi_df['class_resp.keys'][i] == 'period':
            classi_df['glasses_choose'][i] = 1
    
    for i in classi_df.index:
        if '_001' in classi_df['classification_glasses'][i]:
            classi_df['class_glasses'][i] = 0
        elif '_011' in classi_df['classification_glasses'][i]:
            classi_df['class_glasses'][i] = 1
        elif '_021' in classi_df['classification_glasses'][i]:
            classi_df['class_glasses'][i] = 2
        elif '_031' in classi_df['classification_glasses'][i]:
            classi_df['class_glasses'][i] = 3
        elif '_041' in classi_df['classification_glasses'][i]:
            classi_df['class_glasses'][i] = 4
        elif '_051' in classi_df['classification_glasses'][i]:
            classi_df['class_glasses'][i] = 5
        elif '_061' in classi_df['classification_glasses'][i]:
            classi_df['class_glasses'][i] = 6
        elif '_071' in classi_df['classification_glasses'][i]:
            classi_df['class_glasses'][i] = 7
        elif '_081' in classi_df['classification_glasses'][i]:
            classi_df['class_glasses'][i] = 8
        elif '_091' in classi_df['classification_glasses'][i]:
            classi_df['class_glasses'][i] = 9
        elif '_101' in classi_df['classification_glasses'][i]:
            classi_df['class_glasses'][i] = 10
            
    all_simi = all_simi.append(simi_df)
    all_classi = all_classi.append(classi_df)
    all_mani = all_mani.append(mani_df)
    
    #all_simi.to_csv('all_simi.csv', index = False)
    #all_classi.to_csv('all_classi.csv', index = False)
    all_mani.to_csv('all_mani.csv', index = False)

    sub += 1
    #simi_df.to_csv(r'cleaned_data\simi\\\{}.csv'.format(sub), index = False)
    #classi_df.to_csv(r'cleaned_data\classi\\\{}.csv'.format(sub), index = False)
    mani_df.to_csv(r'cleaned_data\mani\\\{}.csv'.format(sub), index = False)

In [3]:
each_df

,pair_assigned,condition,glasses1,glasses2,ID,similarity_resp.keys,similarity_resp.rt,ask_glasses,glasses_left_num,glasses_right_num,classification_glasses,class_resp.keys,class_resp.rt,manipulation_check
0,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,period,1.93325,glasses2,4.0,6.0,NaN,NaN,NaN,NaN
2,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,period,4.28684,glasses2,1.0,3.0,NaN,NaN,NaN,NaN
3,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,x,1.80549,glasses2,9.0,7.0,NaN,NaN,NaN,NaN
4,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,x,2.11255,glasses2,7.0,5.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,NaN,NaN,NaN,NaN,NaN,geometric_square_091.png,period,1.039765,NaN
122,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,NaN,NaN,NaN,NaN,NaN,geometric_square_071.png,period,0.994005,NaN
123,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,NaN,NaN,NaN,NaN,NaN,geometric_square_031.png,x,2.373995,NaN
124,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,NaN,NaN,NaN,NaN,NaN,geometric_square_071.png,period,0.915325,NaN


In [4]:
simi_df

,pair_assigned,condition,glasses1,glasses2,ID,similarity_resp.keys,similarity_resp.rt,ask_glasses,glasses_left_num,glasses_right_num,glasses_choose,correct,pair
1,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,period,1.933250,glasses2,4,6,6,1,pair5
2,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,period,4.286840,glasses2,1,3,3,1,pair2
3,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,x,1.805490,glasses2,9,7,9,1,pair8
4,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,x,2.112550,glasses2,7,5,7,1,pair6
5,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,period,2.259655,glasses2,7,9,9,1,pair8
6,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,period,1.913395,glasses1,1,3,3,0,pair2
7,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,x,2.588305,glasses1,3,1,3,0,pair2
8,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,period,1.761720,glasses1,6,4,4,1,pair5
9,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,period,3.414775,glasses2,2,4,4,1,pair3
10,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,x,1.438460,glasses1,8,10,8,1,pair9


In [5]:
classi_df

,pair_assigned,condition,glasses1,glasses2,ID,classification_glasses,class_resp.keys,class_resp.rt,class_glasses,glasses_choose
37,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_091.png,period,3.765930,9,1
38,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_011.png,x,1.071900,1,0
39,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_051.png,period,0.456345,5,1
40,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_041.png,period,1.730615,4,1
41,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_061.png,period,1.357925,6,1
...,...,...,...,...,...,...,...,...,...,...
120,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_061.png,period,1.110085,6,1
121,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_091.png,period,1.039765,9,1
122,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_071.png,period,0.994005,7,1
123,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_031.png,x,2.373995,3,0


In [6]:
all_classi

,pair_assigned,condition,glasses1,glasses2,ID,classification_glasses,class_resp.keys,class_resp.rt,class_glasses,glasses_choose
38,cateye_oval,face_upright_glasses,cateye,oval,5c07f9ce7ce1dd0001a4efec,cateye_oval_071.png,x,11.619725,7,0
39,cateye_oval,face_upright_glasses,cateye,oval,5c07f9ce7ce1dd0001a4efec,cateye_oval_001.png,x,4.415375,0,0
40,cateye_oval,face_upright_glasses,cateye,oval,5c07f9ce7ce1dd0001a4efec,cateye_oval_021.png,x,1.747670,2,0
41,cateye_oval,face_upright_glasses,cateye,oval,5c07f9ce7ce1dd0001a4efec,cateye_oval_071.png,period,0.773535,7,1
42,cateye_oval,face_upright_glasses,cateye,oval,5c07f9ce7ce1dd0001a4efec,cateye_oval_081.png,period,1.003210,8,1
...,...,...,...,...,...,...,...,...,...,...
120,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_061.png,period,1.110085,6,1
121,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_091.png,period,1.039765,9,1
122,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_071.png,period,0.994005,7,1
123,geometric_square,inverted_glasses,geometric,square,60f85ef4aa5a402b7cf189da,geometric_square_031.png,x,2.373995,3,0
